In [1]:
cd E:/cells/train

E:\cells\train


In [2]:
import pretrainedmodels

In [3]:
import pandas as pd

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import gc
from tqdm import tqdm
import cv2

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [6]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook, trange
import pickle
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score as AUC
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from tqdm import trange
from torch.autograd import Variable
import os
import copy

In [7]:
torch.__version__

'0.4.1'

In [9]:
def plot_history(train_history, val_history, lr):
    plt.style.use('ggplot')
    plt.figure(figsize=[12,6])
    plt.subplot(1,2,1)
    plt.title('Loss')
    plt.plot(train_history, label='train', zorder=1, c='lightseagreen')
    
    points = np.array(val_history)
    
    plt.scatter(points[:, 0], points[:, 1], marker='o', s=80, c='crimson', label='val', zorder=2)
    plt.xlabel('train steps')
    
    plt.legend(loc='best')
    #plt.grid()
    plt.subplot(1,2,2)
    plt.plot(lr, label='lr')
    plt.xlabel('epoch')
    plt.ylabel('lr')
    plt.title('Learning Rate')
    
    
    plt.show()

In [12]:
class SnapshotEns(object):
    def __init__(self, k):
        self.best_loss = np.array([np.inf]*k)
        self.k = k
        self.models = [0]*k

    def train(self, model, loss):
        if np.any(self.best_loss > loss):
            presorted_loss = self.best_loss
            self.best_loss = np.sort(self.best_loss)
            
            self.models = [self.models[z] for z in np.argsort(presorted_loss)]        
            pos = np.where(self.best_loss > loss)[0][-1]

            self.best_loss[pos] = loss            
            self.models[pos] = copy.deepcopy(model.eval())

    def get(self):
        return (self.best_loss, self.models)
    
    def predict(self, X):
        data = Variable(torch.from_numpy(X)).type(torch.FloatTensor).cuda()
        preds = 0
        for i in self.models:
            preds += i.forward(data).cpu().data.numpy().flatten()
        preds /= self.k
        return preds

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os

In [14]:
data_dir = 'E:/cells/train/train'
data_transforms = {
    'fold_0': transforms.Compose([
        transforms.RandomSizedCrop(224),
        #transforms.Scale(224),
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip(),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'fold_1': transforms.Compose([
        transforms.Scale(224),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'fold_2': transforms.Compose([
        transforms.Scale(224),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['fold_0', 'fold_1', 'fold_2']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['fold_0', 'fold_1', 'fold_2']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['fold_0', 'fold_1', 'fold_2']}
class_names = image_datasets['fold_0'].classes
use_gpu = torch.cuda.is_available()

C:\Users\Simakov\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:397: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")
C:\Users\Simakov\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:156: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [15]:
use_gpu = torch.cuda.is_available()

In [18]:
def adj_learning_rate(optimizer, initial_lr,  epoch):
    lr = np.abs(np.sin(np.pi/4 * epoch))*0.02 + 0.02/100
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    print('LR: {l}'.format(l=lr))
    return lr

In [19]:
from sklearn.metrics import f1_score

In [20]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0
    #global se
    #se = SnapshotEns(k=3)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['fold_0', 'fold_1', 'fold_2']:
            if phase == 'fold_0' or phase == 'fold_1':
                #scheduler.step()
                #adj_learning_rate(optimizer=optimizer, initial_lr=0.01, epoch=epoch)
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0

            pred = []
            true = []
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data

                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                true.extend(labels.data.cpu().numpy())
                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                if phase == 'fold_0' or phase == 'fold_1':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)
                pred.extend(outputs[:,1].data.cpu().numpy())
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = roc_auc_score(y_true=true, y_score=pred)
            epoch_f1 = f1_score(average='macro', y_true=true, y_pred= 1* (np.array(pred) > 0.5))
            
            print('{} Loss: {:.4f},  auc: {:.4f}, f1: {:.4f}'.format(
                phase, epoch_loss, epoch_acc, epoch_f1))

            if phase == 'fold_2' and epoch_f1 > best_acc:
                best_acc = epoch_f1
                best_model_wts = model.state_dict()
        
        time_elapsed = time.time() - since
        print('Elapsed {:.0f}m {:.0f}s\n'.format(time_elapsed // 60, time_elapsed % 60))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val auc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [22]:
model_ft = pretrainedmodels.__dict__['resnet34'](num_classes=1000, pretrained='imagenet')

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to C:\Users\Simakov/.torch\models\resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:12<00:00, 6902383.18it/s]


In [24]:
#model_ft.last_linear = nn.Linear(2048, 2)
#model_ft.last_linear = nn.Linear(4320, 2)
model_ft.last_linear = nn.Linear(512, 2)
params_to_train = model_ft.parameters()

if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

optimizer_ft = torch.optim.Adam(params_to_train, lr=0.00001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [25]:
# resnet34
model_ft = train_model(
    model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=30)

Epoch 0/29
----------


100%|██████████| 882/882 [01:16<00:00, 11.46it/s]


fold_0 Loss: 0.1199,  auc: 0.8052, f1: 0.5916


100%|██████████| 896/896 [01:17<00:00, 11.52it/s]


fold_1 Loss: 0.0868,  auc: 0.9018, f1: 0.7254


...

fold_0 Loss: 0.0337,  auc: 0.9857, f1: 0.9216


100%|██████████| 896/896 [01:15<00:00, 11.79it/s]


fold_1 Loss: 0.0021,  auc: 1.0000, f1: 0.9971


100%|██████████| 889/889 [00:17<00:00, 50.36it/s]


fold_2 Loss: 0.1895,  auc: 0.8516, f1: 0.8065
Elapsed 84m 51s

Training complete in 84m 51s
Best val auc: 0.815967


In [30]:
# seresnext50
model_ft = train_model(
    model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=30)

Epoch 0/29
----------


100%|██████████| 882/882 [05:08<00:00,  2.86it/s]


fold_0 Loss: 0.1005,  auc: 0.8743, f1: 0.6856


100%|██████████| 896/896 [05:12<00:00,  2.87it/s]


fold_1 Loss: 0.0706,  auc: 0.9373, f1: 0.8303


...

fold_0 Loss: 0.0249,  auc: 0.9920, f1: 0.9449


100%|██████████| 896/896 [05:12<00:00,  2.87it/s]


fold_1 Loss: 0.0026,  auc: 1.0000, f1: 0.9971


100%|██████████| 889/889 [01:17<00:00, 11.48it/s]


fold_2 Loss: 0.1944,  auc: 0.8940, f1: 0.8134
Elapsed 348m 58s

Training complete in 348m 58s
Best val auc: 0.836964


In [31]:
#torch.save(model_ft.state_dict(), 'se_resnext101_32x4d_v1')

In [24]:
#mods = models.resnet50()
#mods.fc = nn.Linear(2048, 2)

#mods.load_state_dict(torch.load('./se_resnext101_32x4d_v1'))
#mods = mods.cuda()